# Hosting MCP Server on Amazon Bedrock AgentCore Runtime - OAuth Inbound Authentication

## Overview

In this tutorial we will learn how to host MCP (Model Context Protocol) servers on Amazon Bedrock AgentCore Runtime. We will use the Amazon Bedrock AgentCore Python SDK to wrap MCP tools as an MCP server compatible with Amazon Bedrock AgentCore.

The Amazon Bedrock AgentCore Python SDK handles the MCP server implementation details so you can focus on your tools' core functionality. It transforms your code into the AgentCore standardized MCP protocol contracts for direct communication.

### Tutorial Details

| Information         | Details                                                   |
|:--------------------|:----------------------------------------------------------|
| Tutorial type       | Hosting Tools                                             |
| Tool type           | MCP server                                                |
| Tutorial components | Hosting MCP server on AgentCore Runtime                  |
| Tutorial vertical   | Cross-vertical                                            |
| Example complexity  | Easy                                                      |
| SDK used            | Amazon BedrockAgentCore Python SDK and MCP               |

### Tutorial Architecture

In this tutorial we will describe how to deploy an MCP server to AgentCore runtime.

For demonstration purposes, we will use a simple MCP server with 3 tools: `add_numbers`, `multiply_numbers` and `greet_user`

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### Tutorial Key Features

* Creating MCP servers with custom tools
* Testing MCP servers locally
* Hosting MCP servers on Amazon Bedrock AgentCore Runtime
* Invoking deployed MCP servers with authentication


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) library
* Docker running

# Note:
AWS credentials can come from multiple sources, and boto3 follows a very specific search order.
We are getting authenticated because boto3 is finding your credentials in:

`~/.aws/credentials` or `~/.aws/config`

This is the standard location for AWS CLI credentials, and boto3 automatically uses them.

In [1]:
# !pip install --force-reinstall -U -r requirements.txt --quiet

In [2]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os
import sys

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool

sys.path[0]: /home/noone/Documents


In [3]:
boto_session = Session()
region = boto_session.region_name

ssm_client = boto_session.client('ssm', region_name=region)
secrets_client = boto_session.client('secretsmanager', region_name=region)
agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)

tool_name = "mcp_server_agentcore"

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

### Project Structure

This script will generate the following files for our project, with the proper structure:

```
build_club_mcp/
├── mcp_server.py              # Main MCP server code
├── my_mcp_client.py          # Local testing client
├── my_mcp_client_remote.py   # Remote testing client
├── requirements.txt          # Dependencies
└── __init__.py              # Python package marker
```

## Creating MCP Server

Now let's create our MCP server with three simple tools. The server uses FastMCP with `stateless_http=True` which is required for AgentCore Runtime compatibility.

In [4]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """
    Add two numbers together
    :param a:
    :param b:
    :return:
    """
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """
    Multiply two numbers together
    :param a:
    :param b:
    :return:
    """
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """
    Greet a user by name
    :param name:
    :return:
    """
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Overwriting mcp_server.py


### What This Code Does

* **FastMCP**: Creates an MCP server that can host your tools
* **@mcp.tool()**: Decorator that turns your Python functions into MCP tools
* **stateless_http=True**: Required for AgentCore Runtime compatibility
* **Tools**: Three simple tools demonstrating different types of operations

## Creating Local Testing Client

Before deploying to AgentCore Runtime, let's create a client to test our MCP server locally:

In [5]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Overwriting my_mcp_client.py


### Testing Locally

To test your MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp_server.py
   ```
   
2. **Terminal 2**: Run the test client
   ```bash
   python my_mcp_client.py
   ```

You should see your three tools listed in the output.

## Setting up Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server.

In [6]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

Setting up Amazon Cognito user pool...
Pool id: ap-southeast-2_R0oU7wbaB
Discovery URL: https://cognito-idp.ap-southeast-2.amazonaws.com/ap-southeast-2_R0oU7wbaB/.well-known/openid-configuration
Client ID: o8mhr9c7a0q276g7ha58oaj66
Bearer Token: eyJraWQiOiJRZU9qeVdvQitlc05xSVY0REFWMEV4Z1JxcjNyeTB6RXpcLzVyYmNDcTlGcz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJjOTVlODQ2OC1iMGIxLTcwY2UtNWNmZC1lN2U4ZWY4NWMwNGEiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtc291dGhlYXN0LTIuYW1hem9uYXdzLmNvbVwvYXAtc291dGhlYXN0LTJfUjBvVTd3YmFCIiwiY2xpZW50X2lkIjoibzhtaHI5YzdhMHEyNzZnN2hhNThvYWo2NiIsIm9yaWdpbl9qdGkiOiIxYzJhMWU2Yi0xZjM1LTRkZjktYWVkMy1iOTc5MTdjMDM2YjYiLCJldmVudF9pZCI6IjA0MzM1ZGM2LTZlMmEtNGFhNC04NDNhLTE5NTA0Zjc0ZTY1NyIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjUzMjI2MzEsImV4cCI6MTc2NTMyNjIzMSwiaWF0IjoxNzY1MzIyNjMxLCJqdGkiOiJiNzhiYzQwMS04NGQ0LTQ3MzUtYTA4Zi1jZDk1MjNhYzdiY2IiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.gGPfWfJQ4ACQyap2OQK9xM5OUeYlw3vYAVKgdBUWY3bAVotPc

## Note:
these variables are just jupyter variables. They are not persistent. we will save them to AWS Systems Manager Parameter Store and AWS Secrets Manager in a subsequent step

## Configuring AgentCore Runtime Deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [7]:
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'pyproject.toml']  # note: you can use requirements.txt
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="pyproject.toml",  # this needs to match the earlier declaration
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

Entrypoint parsed: file=/home/noone/Documents/GitHub/build_club_mcp/mcp_server.py, bedrock_agentcore_name=mcp_server
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: mcp_server_agentcore
Memory disabled
Network mode: PUBLIC


Using AWS region: ap-southeast-2
All required files found ✓
Configuring AgentCore Runtime...


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/noone/Documents/GitHub/build_club_mcp/.dockerignore
Keeping 'mcp_server_agentcore' as default agent
Bedrock AgentCore configured: /home/noone/Documents/GitHub/build_club_mcp/.bedrock_agentcore.yaml


Configuration completed ✓


⚠️ **Note about CPU architecture**
Bedrock AgentCore is designed to run on `linux/arm64` (Graviton).
If your laptop has an x86 CPU (Intel/AMD), you may see a **platform mismatch warning**
when using local Docker builds. This does **not** affect the remote runtime in AWS.

We will use the **managed remote build** for this workshop, which works on all laptops.

The next section shows an **optional workaround** to run the container locally on x86_64
using Docker Buildx and QEMU emulation.

🧪 This is **not covered in the workshop**. Proceed at your own risk — we won’t have time
to debug Docker build issues on individual laptops. If you do choose to proceed, uncomment the relevant cell below, and consider running the commands in the CLI instead of from the notebook

In [8]:
# 1. Install QEMU/binfmt for multi-arch emulation
# !docker run --privileged --rm tonistiigi/binfmt --install all

# 2. Create a multi-arch buildx builder
# !docker buildx create --name multiarch --use
# !docker buildx inspect --bootstrap

# 3. Build the image for arm64
# !docker buildx build \
#   --platform linux/arm64 \
#   -t mcp-server-agentcore:local .

# 4. Run it with emulation
# !docker run \
#   --platform linux/arm64 \
#   -p 8080:8080 \
#   mcp-server-agentcore:local

Lets add some policies:

In [9]:
# import boto3
# from boto3.session import Session
#
# print("Granting ECR read access to the AgentCore runtime role...")
#
# session = Session()
# region = session.region_name or "ap-southeast-2"
# iam = session.client("iam")
#
# # The starter toolkit creates a role with this prefix:
# role_prefix = f"AmazonBedrockAgentCoreSDKRuntime-{region}-"
#
# # Find the first matching runtime role
# paginator = iam.get_paginator("list_roles")
# runtime_role_name = None
#
# for page in paginator.paginate():
#     for role in page["Roles"]:
#         if role["RoleName"].startswith(role_prefix):
#             runtime_role_name = role["RoleName"]
#             break
#     if runtime_role_name:
#         break
#
# if not runtime_role_name:
#     raise RuntimeError(
#         f"Could not find AgentCore runtime role with prefix: {role_prefix}"
#     )
#
# print(f"Found runtime role: {runtime_role_name}")
#
# # Attach a managed policy that gives ECR read access
# iam.attach_role_policy(
#     RoleName=runtime_role_name,
#     PolicyArn="arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryReadOnly",
# )
#
# print("✓ Attached AmazonEC2ContainerRegistryReadOnly to runtime role")
# print("You can now safely run agentcore_runtime.launch()")

This:
- Uses your current AWS profile / region (same as the rest of the notebook).
- Locates the auto-created runtime role by name prefix.
- Attaches a standard AWS managed policy that includes:
    `ecr:GetAuthorizationToken`
    `ecr:BatchGetImage`
    `ecr:GetDownloadUrlForLayer`

Once that’s done, the subsequent `agentcore_runtime.launch()` call will pass

## Launching MCP Server to AgentCore Runtime

Now that we've got a docker file, let's launch the MCP server to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [10]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()  # this is the important bit
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 001499655372 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: mcp_server_agentcore


Launching MCP server to AgentCore Runtime...
This may take several minutes...


ECR repository available: 001499655372.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-mcp_server_agentcore
Getting or creating execution role for agent: mcp_server_agentcore
Using AWS region: ap-southeast-2, account ID: 001499655372
Role name: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4


✅ Reusing existing ECR repository: 001499655372.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-mcp_server_agentcore


✅ Reusing existing execution role: arn:aws:iam::001499655372:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4
Execution role available: arn:aws:iam::001499655372:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: mcp_server_agentcore
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-7f3ae149b4
Reusing existing CodeBuild execution role: arn:aws:iam::001499655372:role/AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-7f3ae149b4
Using dockerignore.template with 46 patterns for zip filtering
Uploaded source to S3: mcp_server_agentcore/source.zip
Updated CodeBuild project: bedrock-agentcore-mcp_server_agentcore-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 7.3s
🔄 DOWNLOAD_SOU

ValidationException: An error occurred (ValidationException) when calling the CreateAgentRuntime operation: Access denied while validating ECR URI '001499655372.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-mcp_server_agentcore:latest'. The execution role requires permissions for ecr:GetAuthorizationToken, ecr:BatchGetImage, and ecr:GetDownloadUrlForLayer operations.

# Note:
lets read the output. you'll see that AgentCore has created:
- roles
- execution policy
- source uploaded to s3 and CodeBuild project created
- docker image build & pushed via CodeBuild

-## 🎉 Bedrock AgentCore Deployment Successful

The code above has **successfully built and deployed** the MCP server to **Amazon Bedrock AgentCore Runtime** using CodeBuild and the AgentCore SDK.

Here’s what happened step-by-step:

1. **Source code was packaged** using your `.dockerignore` template.
2. **CodeBuild** built an ARM64 image and pushed it to **ECR**.
3. The AgentCore Runtime was **updated with the new container image**.
4. AWS automatically configured:
   - CloudWatch Logs
   - X-Ray tracing
   - GenAI Observability Dashboard
5. The runtime endpoint became **available and ready to receive requests**.

---

## 🔑 Important: Save Your Agent Identifiers

You will need these values for:
- invoking the agent,
- connecting via Strands,
- debugging,
- or integrating with other systems / MCP clients.

**Agent ID:** <your agent ID>

**Agent ARN:**  <your agent ARN>

If you are regenerating or redeploying frequently, these values may change — so always use the **current** ones shown in the launch output.

## Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN and Cognito configuration in AWS Systems Manager Parameter Store and AWS Secrets Manager for easy retrieval:

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## Creating Remote Testing Client

Now let's create a client to test our deployed MCP server. This client will retrieve the necessary credentials from AWS and connect to the deployed server:

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import base64
import json
import sys
import time
import traceback
from datetime import timedelta
from typing import Optional

import boto3
from boto3.session import Session

def get_refresh_token(client_id: str, refresh_token: str, region: str) -> str:
    """
    Refresh access token using refresh token
    :param client_id: The Cognito User Pool App Client ID
    :param refresh_token: The persistent Cognito refresh token associated with the user/session
    :param region: The AWS region where the Cognito User Pool lives (e.g. "ap-southeast-2")
    :return: A newly issued Cognito access token
    :raises: `botocore.exceptions.BotoCoreError` If there is a low-level error calling Cognito.
    `botocore.exceptions.ClientError` If Cognito rejects the refresh request (e.g. invalid token, client, or user)
    """
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']


def get_valid_token(bearer_token: str, client_id: str, refresh_token: str, region: str):
    """
    Check token expiry and refresh if needed
    :param bearer_token: The current access token (JWT string) used as a Bearer token.
    :param client_id: The Cognito User Pool App Client ID.
    :param refresh_token: The long-lived Cognito refresh token.
    :param region: AWS region of the Cognito User Pool.
    :return: A valid access token (either the original or a newly refreshed one)
    """
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except Exception as e:
        print("🔄 Invalid token, refreshing...", e)
        traceback.print_exc()
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    """
    Entry point for connecting to a remote Bedrock AgentCore MCP runtime.
    This function performs the following steps:
    1. Resolve the AWS region from the current boto3 session.
    2. Retrieve the **Agent ARN** from AWS Systems Manager Parameter Store
       (`/mcp_server/runtime/agent_arn`).
    3. Retrieve Cognito credentials (bearer token, refresh token, client ID)
       from AWS Secrets Manager (`mcp_server/cognito/credentials`).
    4. Validate or refresh the bearer token as needed.
    5. Construct the AgentCore MCP HTTPS invocation URL.
    6. Open a stream to the MCP runtime using `streamablehttp_client`.
    7. Create an `ClientSession`, initialize the MCP connection, and list
       available tools exposed by the remote MCP server.

    On success, it prints a table-like list of tools and their descriptions.

    The process exits with a non-zero status code if:
      - the agent ARN cannot be retrieved,
      - credentials cannot be retrieved, or
      - the MCP connection fails.
    """
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

# What it Does
This script connects to your deployed AgentCore MCP server using Cognito auth, automatically refreshes tokens, and builds the correct HTTPS invocation URL. It opens a streaming MCP session, initializes it, and lists all available tools. Run it after deployment to confirm your cloud runtime is reachable and working end-to-end.

## Testing Your Deployed MCP Server

Let's test our deployed MCP server using the remote client:

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

## Invoking MCP Tools Remotely

Now let's create an enhanced client that not only lists tools but also invokes them to demonstrate the full MCP functionality:

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client


def get_refresh_token(client_id: str, refresh_token: str, region: str) -> str:
    """
    Refresh an access token using a long-lived Cognito refresh token.
    :param client_id: Cognito app client ID for the user pool.
    :param refresh_token: Long-lived refresh token issued by Cognito.
    :param region: AWS region where the Cognito user pool resides.
    :returns: A fresh short-lived bearer access token.
    """
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']


def get_valid_token(bearer_token: str, client_id: str, refresh_token: str, region: str) -> str:
    """
    Validate a bearer token and refresh it if it is invalid or close to expiry.
    :param bearer_token: Current bearer access token (JWT).
    :param client_id: Cognito app client ID for the user pool.
    :param refresh_token: Long-lived refresh token issued by Cognito.
    :param region: AWS region where the Cognito user pool resides.
    :returns: A valid bearer token, refreshed if necessary.
    """
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except:
        print("🔄 Invalid token, refreshing...")
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

# What this cell does
This script connects to your deployed MCP server using its Agent ARN, refreshes Cognito tokens, and opens a remote MCP session. It lists all available tools and then invokes several sample tools to validate end-to-end functionality. Use this to confirm your AgentCore deployment is working correctly.

## Test Tool Invocation

Let's test our MCP tools by actually invoking them:

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## Next Steps

Now that you have successfully deployed an MCP server to AgentCore Runtime, you can:

1. **Add More Tools**: Extend your MCP server with additional tools
2. **Add Authentication**: Implement Oauth2 or JWT authorizers
3. **Integration**: Integrate with other AgentCore services

## Cleanup (Optional)

If you want to clean up the resources created during this tutorial, run the following cells:

In [ ]:
# print("🗑️  Starting cleanup process...")
#
# try:
#     ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")
#
# try:
#     secrets_client.delete_secret(
#         SecretId='mcp_server/cognito/credentials',
#         ForceDeleteWithoutRecovery=True
#     )
#     print("✓ Secrets Manager secret deleted")
# except secrets_client.exceptions.ResourceNotFoundException:
#     print("ℹ️  Secrets Manager secret not found")
#
# print("\n✅ Cleanup completed successfully!")

In [ ]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )

### Cleanup Successful

Your MCP AgentCore deployment and all supporting resources have been fully removed:
- Agent runtime & endpoint
- ECR images and repository
- CodeBuild project & IAM roles
- Deployment artifacts in S3
- Local AgentCore configuration

You now have a clean AWS environment and can safely re-run the workshop from the start.

# 🎉 Congratulations!

You have successfully:

✅ **Created an MCP server** with custom tools  
✅ **Tested locally** using MCP client  
✅ **Set up authentication** with Amazon Cognito  
✅ **Deployed to AWS** using AgentCore Runtime  
✅ **Invoked remotely** with proper authentication  
✅ **Learned MCP concepts** and best practices  

Your MCP server is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

## Summary

In this tutorial, you learned how to:
- Build MCP servers using FastMCP
- Configure stateless HTTP transport for AgentCore compatibility
- Set up JWT authentication with Amazon Cognito
- Deploy and manage MCP servers on AWS
- Test both locally and remotely
- Use MCP clients for tool invocation

The deployed MCP server can now be integrated into larger AI applications and workflows!